In [1]:
from ReflexCtrInterface import MyoLegReflex
import numpy as np
import skvideo.io

# create reflex object (it creates its own myoLegStandRandom env internally)
reflex = MyoLegReflex()
reflex.reset()

# load baseline params that come with the tutorial
params = np.loadtxt("baseline_params.txt")
reflex.set_control_params(params)

frames = []
for step in range(200):                # run ~2 seconds at dt=0.01
    out_dict, is_done, sim_time, new_act = reflex.run_reflex_step()
    frames.append(reflex.env.sim.renderer.render_offscreen(width=640, height=480, camera_id=1))
    if is_done:
        break

skvideo.io.vwrite("myoleg_reflex_demo.mp4", np.asarray(frames), outputdict={"-pix_fmt":"yuv420p"})
print("Saved myoleg_reflex_demo.mp4")


MyoSuite:> Registering Myo Envs


x:\Deep Reinforcement Learning\.venv\Lib\site-packages\gymnasium\envs\registration.py:694: UserWarning: WARN: Overriding environment myoArmReachFixed-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
x:\Deep Reinforcement Learning\.venv\Lib\site-packages\gymnasium\envs\registration.py:694: UserWarning: WARN: Overriding environment myoSarcArmReachFixed-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
x:\Deep Reinforcement Learning\.venv\Lib\site-packages\gymnasium\envs\registration.py:694: UserWarning: WARN: Overriding environment myoFatiArmReachFixed-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
x:\Deep Reinforcement Learning\.venv\Lib\site-packages\gymnasium\envs\registration.py:694: UserWarning: WARN: Overriding environment myoArmReachRandom-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in reg

    MyoSuite: A contact-rich simulation suite for musculoskeletal motor control
        Vittorio Caggiano, Huawei Wang, Guillaume Durandau, Massimo Sartori, Vikash Kumar
        L4DC-2019 | https://sites.google.com/view/myosuite
    
Seed added -  0


x:\Deep Reinforcement Learning\.venv\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.seed to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.seed` for environment variables or `env.get_wrapper_attr('seed')` that will search the reminding wrappers.
  logger.warn(
x:\Deep Reinforcement Learning\.venv\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.sim to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.sim` for environment variables or `env.get_wrapper_attr('sim')` that will search the reminding wrappers.
  logger.warn(


Saved myoleg_reflex_demo.mp4


In [3]:
from ReflexCtrInterface import MyoLegReflex
import numpy as np
import skvideo.io
from myosuite.utils import gym

In [4]:
soccer_env = gym.make("myoChallengeSoccerP1-v0", normalize_act=False)

In [5]:
init_dict = MyoLegReflex.DEFAULT_INIT_POSE.copy()
init_dict['env'] = soccer_env
reflex = MyoLegReflex(init_dict=init_dict)
reflex.reset()
obs = soccer_env.reset()

Seed added -  0


In [6]:
def reflex_to_soccer_ctrl(reflex, soccer_env):
    """
    Get current reflex stim and produce a ctrl vector compatible with soccer_env.
    - reflex: instance of MyoLegReflex (from ReflexCtrInterface)
    - soccer_env: gym env (myoChallengeSoccerP1-v0)
    Returns: numpy array shape matching soccer_env.sim.data.ctrl
    """
    # 1) get reflex stim (dict) and convert to leg-only mus_act (80-length)
    stim_dict = reflex.update()                     # returns dict: {'r_leg': {...}, 'l_leg': {...}}
    mus_act_leg = reflex.reflex2mujoco(stim_dict)   # length 80 (specific to the leg-only actuator indexing)

    # 2) prepare full ctrl vector for the soccer env
    ctrl_full = np.zeros_like(soccer_env.sim.data.ctrl)  # length = soccer_env.sim.data.ctrl.shape[0]

    # 3) map values from the leg-reflex environment indexing to the soccer env actuator indices
    #    We'll use actuator NAMES (muscle labels) to find corresponding indices in both envs.
    #    reflex.muscle_labels: nested dict: {'r_leg': {'GLU': [...], ...}, 'l_leg': {...}}
    #    We'll iterate muscle label names and copy the corresponding value.
    for s_leg in ['r_leg', 'l_leg']:
        for group_name, name_list in reflex.muscle_labels[s_leg].items():
            for muscle_name in name_list:
                # index in the reflex.env actuator array
                try:
                    idx_reflex = reflex.env.sim.model.actuator(muscle_name).id
                except Exception as e:
                    # if reflex.env does not have that muscle name, skip
                    # (should not happen for standard MyoLeg)
                    continue

                # index in the soccer env actuator array (may be different)
                try:
                    idx_soccer = soccer_env.sim.model.actuator(muscle_name).id
                except Exception:
                    # soccer env may not expose certain actuators with exactly the same name;
                    # skip those safely (or print for debugging)
                    # print("missing actuator in soccer env:", muscle_name)
                    continue

                # copy value (value in mus_act_leg is placed using reflex.env indexing)
                ctrl_full[idx_soccer] = mus_act_leg[idx_reflex]

    return ctrl_full


In [ ]:
# assume you already did:
# reflex.reset()
# soccer_env.reset()

ctrl = reflex_to_soccer_ctrl(reflex, soccer_env)
print("ctrl shape expected:", soccer_env.sim.data.ctrl.shape, "got:", ctrl.shape)
# step
obs, reward, terminated, truncated, info = soccer_env.step(ctrl)
print("step OK, reward:", reward, "terminated:", terminated, "truncated:", truncated)


ctrl shape expected: (290,) got: (290,)
step OK, reward: -1.4089739909912875 terminated: False truncated: False


x:\Deep Reinforcement Learning\.venv\Lib\site-packages\gymnasium\utils\passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


In [12]:
for _ in range(20000):
    stim_dict = reflex.update()
    muscle_act = reflex.reflex2mujoco(stim_dict)
    # step the soccer env with the reflex output
    obs, reward, terminated, truncated, info = soccer_env.step(ctrl)
    frame = soccer_env.sim.renderer.render_offscreen(width=640, height=480, camera_id=1)
    # record or inspect r, info, frame
    if truncated or terminated:
        break
    
skvideo.io.vwrite("myoleg_soccer_reflex_demo.mp4", np.asarray([frame]), outputdict={"-pix_fmt":"yuv420p"})
print("Saved myoleg_soccer_reflex_demo.mp4")

Saved myoleg_soccer_reflex_demo.mp4


In [15]:
# print reflex actuator names and ids
print("Reflex actuators:")
for i in range(reflex.env.sim.model.nu):   # or len(reflex.env.sim.model.actuator_names)
    print(i, reflex.env.sim.model.actuator(i))

print("Soccer actuators:")
for i in range(soccer_env.sim.model.nu):
    print(i, soccer_env.sim.model.actuator(i))


Reflex actuators:
0 <_MjModelActuatorViews
  acc0: array([0.3357])
  actadr: array([0])
  actlimited: array([0], dtype=uint8)
  actnum: array([1])
  actrange: array([0., 0.])
  biasprm: array([3.8190e-01, 1.4620e+00, 6.0462e+02, 1.0000e+00, 6.6980e-02,
       2.9961e+00, 1.0000e+01, 1.3421e+00, 1.4000e+00, 0.0000e+00])
  biastype: array([2])
  cranklength: array([0.])
  ctrllimited: array([1], dtype=uint8)
  ctrlrange: array([0., 1.])
  dynprm: array([0.01, 0.04, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ])
  dyntype: array([4])
  forcelimited: array([0], dtype=uint8)
  forcerange: array([0., 0.])
  gainprm: array([3.8190e-01, 1.4620e+00, 6.0462e+02, 1.0000e+00, 6.6980e-02,
       2.9961e+00, 1.0000e+01, 1.3421e+00, 1.4000e+00, 0.0000e+00])
  gaintype: array([2])
  gear: array([1., 0., 0., 0., 0., 0.])
  group: array([0])
  id: 0
  length0: array([0.1402])
  lengthrange: array([0.0748, 0.1862])
  name: 'addbrev_r'
  trnid: array([ 0, -1])
  trntype: array([3])
  user: array([], dty